<a href="https://colab.research.google.com/github/neelsoumya/butterfly_detector/blob/master/deepaugment_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial for DeepAugment

- Github: https://github.com/barisozmen/deepaugment
- Slides: https://bit.ly/deepaugmentslides
- PyPI: https://pypi.org/project/deepaugment/


In this tutorial, we will:
1. Install
2. Import
3. Setup and configure
4. Optimizer augmentations
5. Print found augmentation policies
6. Create image-generator using augmentation policies
7. Use the image-generator on a full-model

**Important Note:** This notebook is only for showing usage of the library. Configurations are made as running the notebook will take 10-20 minutes (using TPU). Therefore this is not a proper analysis with useful results. A proper analysis would take 5-10 hours. For a proper analysis, change configurations as following:
* **child_epochs:** +100 (Section 3)
* **optimizer iterations:** +100 (Section 4)
* **model:** wrn_28_10 or InceptionV3 (Section 7)

See [slides](http://bit.ly/deepaugmentslides) for results with CIFAR-10 and iNaturalist images, where DeepAugment was ran for +4 hours on AWS p3.x2large instance

### Contact
Baris Ozmen, hbaristr@gmail.com

## 1. Install

In [0]:
!pip install deepaugment

    100% |████████████████████████████████| 81kB 7.3MB/s 
    100% |████████████████████████████████| 51kB 20.5MB/s 
    100% |████████████████████████████████| 8.9MB 3.2MB/s 
    100% |████████████████████████████████| 13.9MB 2.8MB/s 
    100% |████████████████████████████████| 83.1MB 373kB/s 
    100% |████████████████████████████████| 645kB 17.8MB/s 
    100% |████████████████████████████████| 12.9MB 3.9MB/s 
    100% |████████████████████████████████| 573kB 21.0MB/s 
    100% |████████████████████████████████| 3.1MB 10.7MB/s 
albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.7 which is incompatible.
  Found existing installation: numpy 1.14.6
    Uninstalling numpy-1.14.6:
      Successfully uninstalled numpy-1.14.6
  Found existing installation: Keras-Applications 1.0.7
    Uninstalling Keras-Applications-1.0.7:
      Successfully uninstalled Keras-Applications-1.0.7
  Found existing installation: pandas 0.22.0
    Uninstalling pandas-0.22.0:
 

### Installed version

In [0]:
!pip freeze | grep deepaugment

deepaugment==1.1.2


## 2.Import

In [0]:
from deepaugment.deepaugment import DeepAugment

Using TensorFlow backend.


## 3. A simple setup of DeepAugment

Number of child epochs are set as 10 in order to make a quick run. For a thorough analysis, set `child_epochs` >=50

In [0]:
from keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# child_epochs set to 10 for a quick run, but it should be >=50 for a proper analysis
my_config = {
    "child_epochs":2,
    "opt_samples":1
}

# X_train.shape -> (N, M, M, 3)
# y_train.shape -> (N)
deepaug = DeepAugment(images=x_train, labels=y_train, config=my_config)

170500096/170498071 [==============================] - 18s 0us/step
Using 2000 training images
BasicCNN model built as child model.
 Model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_2 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 15, 15, 32)        0    

## 4. Optimize for 1 iteration (normally 100-300 iterations needed)
- Each iteration takes ~30 secs on AWS p3.x2large (V100 GPU), therefore a proper optimization with 300 iterations would take ~2.5 hours and cost ~8$.

    - AWS instance p3.x2large cost: 3.09$/h  

In [0]:
# number of iterations set to 2 (set it >=100 for a proper analysis)
best_policies = deepaug.optimize(2)

trial: 1 
 ['gamma-contrast', 0.8442657485810175, 'coarse-salt-pepper', 0.8472517387841256, 'brighten', 0.38438170729269994, 'translate-y', 0.056712977317443194, 'translate-y', 0.47766511732135, 'add-to-hue-and-saturation', 0.47997717237505744, 'emboss', 0.8360787635373778, 'sharpen', 0.6481718720511973, 'emboss', 0.9571551589530466, 'rotate', 0.8700872583584366]
load_pre_augment_weights()'s runtime:  0.0159 sec.
Train on 12000 samples, validate on 1000 samples
Epoch 1/2
 - 76s - loss: 2.2569 - acc: 0.1425 - val_loss: 2.4308 - val_acc: 0.1550
Epoch 2/2
 - 76s - loss: 2.1583 - acc: 0.1928 - val_loss: 1.9824 - val_acc: 0.2570
fit()'s runtime:  151.9835 sec.
1, 0.794, ['gamma-contrast', 0.8442657485810175, 'coarse-salt-pepper', 0.8472517387841256, 'brighten', 0.38438170729269994, 'translate-y', 0.056712977317443194, 'translate-y', 0.47766511732135, 'add-to-hue-and-saturation', 0.47997717237505744, 'emboss', 0.8360787635373778, 'sharpen', 0.6481718720511973, 'emboss', 0.9571551589530466, '

## 5. See found best policies

In [0]:
best_policies

,trial_no,A_aug1_type,A_aug1_magnitude,A_aug2_type,A_aug2_magnitude,B_aug1_type,B_aug1_magnitude,B_aug2_type,B_aug2_magnitude,C_aug1_type,...,D_aug1_type,D_aug1_magnitude,D_aug2_type,D_aug2_magnitude,E_aug1_type,E_aug1_magnitude,E_aug2_type,E_aug2_magnitude,mean_late_val_acc,expected_accuracy_increase(%)
0,0,rotate,0.000,rotate,0.000,rotate,0.000,rotate,0.000,rotate,...,rotate,0.000,rotate,0.000,rotate,0.000,rotate,0.00,0.392,0.0
4,2,dropout,0.801,coarse-dropout,0.679,fog,0.582,coarse-dropout,0.759,rotate,...,translate-x,0.135,sharpen,0.150,translate-x,0.386,horizontal-flip,0.45,0.255,-13.7
2,1,gamma-contrast,0.844,coarse-salt-pepper,0.847,brighten,0.384,translate-y,0.057,translate-y,...,emboss,0.836,sharpen,0.648,emboss,0.957,rotate,0.87,0.206,-18.6


## 6. Create Image Generator by found best policies

In [0]:
import keras
image_gen = deepaug.image_generator_with_top_policies(x_train, keras.utils.to_categorical(y_train))

## 7. Use DeepAugmented images on your model

Let's try  BasicCNN (Child CNN) model which can be created by modules in the package (assuming that your full model is same with BasicCNN). For a proper analysis use WideResNet-28-10, or better, your own model.

In [0]:
from deepaugment.childcnn import ChildCNN
import logging

cnn_config = {"model":"basicCNN", "logging":logging}
full_model = ChildCNN(input_shape=x_train.shape[1:], num_classes=10, config=cnn_config)

BasicCNN model built as child model.
 Model summary:
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 32, 32, 32)        896       
_________________________________________________________________
activation_7 (Activation)    (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 32)        9248      
_________________________________________________________________
activation_8 (Activation)    (None, 30, 30, 32)        0         
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)      

In [0]:
BATCH_SIZE = 64

full_model.model.fit_generator(
    image_gen,
    validation_data=(x_test, keras.utils.to_categorical(y_test)),
    steps_per_epoch=len(x_train) // BATCH_SIZE,
    epochs=5,
    shuffle=True,
    verbose=2
)

Epoch 1/5
Policies are:
[{'A_aug1_type': 'rotate', 'A_aug1_magnitude': 0.0, 'A_aug2_type': 'rotate', 'A_aug2_magnitude': 0.0, 'B_aug1_type': 'rotate', 'B_aug1_magnitude': 0.0, 'B_aug2_type': 'rotate', 'B_aug2_magnitude': 0.0, 'C_aug1_type': 'rotate', 'C_aug1_magnitude': 0.0, 'C_aug2_type': 'rotate', 'C_aug2_magnitude': 0.0, 'D_aug1_type': 'rotate', 'D_aug1_magnitude': 0.0, 'D_aug2_type': 'rotate', 'D_aug2_magnitude': 0.0, 'E_aug1_type': 'rotate', 'E_aug1_magnitude': 0.0, 'E_aug2_type': 'rotate', 'E_aug2_magnitude': 0.0}, {'A_aug1_type': 'dropout', 'A_aug1_magnitude': 0.801, 'A_aug2_type': 'coarse-dropout', 'A_aug2_magnitude': 0.679, 'B_aug1_type': 'fog', 'B_aug1_magnitude': 0.582, 'B_aug2_type': 'coarse-dropout', 'B_aug2_magnitude': 0.759, 'C_aug1_type': 'rotate', 'C_aug1_magnitude': 0.474, 'C_aug2_type': 'shear', 'C_aug2_magnitude': 0.737, 'D_aug1_type': 'translate-x', 'D_aug1_magnitude': 0.135, 'D_aug2_type': 'sharpen', 'D_aug2_magnitude': 0.15, 'E_aug1_type': 'translate-x', 'E_aug1_